# Import des librairies 

In [1]:
import pandas as pd 
from IPython.display import clear_output
import requests
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import re
import csv
import numpy as np
import re
from tqdm import tqdm
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from concurrent import futures
import dateparser

import warnings
warnings.filterwarnings('ignore')

data_path="Data/"#Chemin du dossier pour enregistrer le fichier csv

In [2]:
def check_nan(df):
    for i in df.columns.tolist():
        print("Valeurs nan dans "+str(i)+" : "+str(df[i].isna().sum()))
        
def check_unique(df):
    for i in df.columns.tolist():
        print("Valeurs uniques dans "+str(i)+" : "+str(df[i].nunique()))

# Chargement des données déjà existantes

On charge le dataset avec les données de la page home pour ne pas scrapper que les films qui ont une note moyenne spectateurs

In [3]:
df_home = pd.read_csv(data_path+"allocine_home_cleaned.csv")
df_home.head()

,id,titre,date_sortie,support,duree,genres,synopsis,note_moyenne_presse,note_moyenne_spectateurs
0,178014,avatar : la voie de l'eau,2022-12-14,en salle,192.0,"['Science fiction', 'Aventure', 'Fantastique',...",Se déroulant plus d’une décennie après les évé...,"4,1","4,3"
1,281293,les banshees d'inisherin,2022-12-28,en salle,114.0,['Drame'],Sur Inisherin - une île isolée au large de la ...,"3,8","3,9"
2,289305,tempête,2022-12-21,en salle,109.0,"['Comédie dramatique', 'Famille']","Née dans le haras de ses parents, Zoé a grandi...","3,0","3,9"
3,266320,m3gan,2022-12-28,en salle,102.0,"['Epouvante-horreur', 'Thriller']","M3GAN est un miracle technologique, une cyber ...","2,9","2,9"
4,288544,le tourbillon de la vie,2022-12-21,en salle,121.0,['Drame'],Les grands tournants de notre existence sont p...,"3,3","3,9"


# Scrapping des données de la page commentaires pour chaque films

## Fonctions

In [27]:
#Renvoie une liste de dictionnaires contenant les données pour chaque review présente sur la page. 
#Si pas de commentaires sur la page renvoie une liste vide
def get_data_reviews_page(id_movie, num_page) :
    url = f"https://www.allocine.fr/film/fichefilm-{id_movie}/critiques/spectateurs/?page={num_page}"
    data=[]
    req = Request(
        url=url, 
        headers={'User-Agent': 'Mozilla/5.0'}
    )
    try :
        webpage = urlopen(req).read()
        soup = BeautifulSoup(webpage, 'html.parser')
        sections = soup.find_all("section",{"class" :"section" }) 

        if len(sections) !=0 :  
            reviews=  sections[0].find_all("div",{"class" : "hred review-card cf"})
            if len(reviews)!= 0 : 
                for index,review in enumerate(reviews) : 
                    data_review ={"id_movie":id_movie}
                    thumbnails = review.find_all("span",{"class":"thumbnail-container"})
                    divs_note=  review.find_all("span",{'class':'stareval-note'})
                    date_container = review.find_all("span" , {"class":"review-card-meta-date light"})
                    div_contenu = review.find_all("div", {"class" : "content-txt review-card-content"})

                    if len(thumbnails) !=0 : 
                        data_review["pseudo"] = thumbnails[0].get("title")
                    if len(divs_note) !=0 :
                        data_review["note"] =divs_note[0].text
                    if len(date_container) !=0 :
                        try :
                            data_review["date"] =re.search(r"(?i)Publiée le (\d{1,2} \w+ \d{4})", date_container[0].text).group(1)
                        except : 
                            print(f"id : {id_movie}, page : {num_page}, review : {index}, pas de date" )
                    if len(div_contenu) !=0 : 
                        all_text = div_contenu[0].text
                        text_no_spoils =all_text 
                        spoilers = div_contenu[0].find_all("span",{"class":"spoiler-content"})
                        if len(spoilers)>0:
                            for spoil in spoilers : 
                                text_no_spoils= text_no_spoils.replace(spoil.text,"\n")
                        data_review["contenu_complet"] = all_text
                        data_review["contenu_sans_spoils"]=text_no_spoils

                    data.append(data_review)
    except Exception as err :
        print(f'id :{id_movie}, page :{num_page}, erreur : {err}')
          
    return data

In [20]:
#inputs : id du film, nombre de pages maximal que l'on souhaite scrapper
#output
def reviews_movies(id_movie, num_pages) : 
    data=[] #
    num_page= 1
    while True : 
        data_returned = get_data_reviews_page(id_movie, num_page)
        
        #On vérifie que des données sont retournées
        if len(data_returned) == 0 :
            break      
        #On vérifie que l'on ne scrappe pas 2 fois les données de la dernière page
        if len(data) !=0 and data[-1]["contenu_complet"] == data_returned[-1]["contenu_complet"]: 
            break

        data+=data_returned
        num_page+=1
        if num_page==num_pages: 
            break
            
    
    return data 

In [21]:
def init_save_files(number) :
    for i in range(number) :
        df_init= pd.DataFrame(columns= ['id_movie','pseudo','note','date','contenu_complet','contenu_sans_spoils'])
        df_init.to_csv(data_path+f"allocine_reviews/reviews_{i}.csv",index=False)

In [22]:
def init_movies_by_driver(nb_drivers, len_data) :
    drivers_ids_range ={}
    nb_films_by_driver = np.ceil(len_data/nb_drivers)
    for i in range(nb_drivers) :
        drivers_ids_range[i]=  {"start" : int(i * nb_films_by_driver), "end" : int((i+1)*nb_films_by_driver)}
    return drivers_ids_range 

### Idée mais finalement non retenue : 

In [23]:
# Les films étant triés par popularité décroissante, les premiers films ont beaucoup plus de commentaires que les derniers
#Il est donc plus long de scrapper les données des premiers films que des derniers 
#Pour que les différents threads soient actifs pendant une période de temps plus équilibrée nous attribuons plus de films 
#aux derniers threads qu'aux premiers.

def compute_factor(index,number):
    return (index+np.sum(z for z in range(0,index)))/np.sum(i for i in range(1,number+1))

def init_movies_by_threads(nb_threads, len_data) :
    threads_ids_range ={}        
    for j in range(0,nb_threads) : 
        threads_ids_range[j] = {"start" : int(compute_factor(j,nb_threads)* len_data), 
                                "end" : int(compute_factor(j+1,nb_threads)* len_data)}
    return threads_ids_range 

#### Test fonction 

In [24]:
init_movies_by_threads(8, 20000)

{0: {'start': 0, 'end': 555},
 1: {'start': 555, 'end': 1666},
 2: {'start': 1666, 'end': 3333},
 3: {'start': 3333, 'end': 5555},
 4: {'start': 5555, 'end': 8333},
 5: {'start': 8333, 'end': 11666},
 6: {'start': 11666, 'end': 15555},
 7: {'start': 15555, 'end': 20000}}

On observe que le premier threads va scrapper 555 films contre 4445 pour le dernier.

Après avoir essayé cette répartition en conditions réelles, il semblerait que le threads 0 est plus efficace que les autres, nous choisissons donc de ne pas la retenir. Nous décidons à la place de mélanger le dataset d'origine.

In [25]:
def workload_threads(params) : 
    ids_to_process = list_ids[params["index_start"]:params["index_end"]]
    new_data = []
    
    for index, id in enumerate(ids_to_process) : 
        new_data+= reviews_movies(id, params["num_pages"])
        if index % 10 == 0 : 
            if params["id"]==0 : 
                clear_output()
            print(f"Threads n°{params['id']} : {index} films sur {len(ids_to_process)}")
    
    #Sauvegarde des données : 
    df= pd.DataFrame(new_data)
    temp = pd.read_csv(data_path+f"allocine_reviews/reviews_{params['id']}.csv")
    pd.concat([temp,df]).to_csv(data_path+f"allocine_reviews/reviews_{params['id']}.csv",index=False)

## Proto

Table commentaires : 

    
id_movie, pseudo auteur , date de publication , nombre d'étoiles, contenu 


## Test fonction

In [11]:
%%time
data = reviews_movies(281293, 20)
df = pd.DataFrame(data)
check_nan(df)
print()
check_unique(df)
df.head()

Valeurs nan dans id_movie : 0
Valeurs nan dans pseudo : 0
Valeurs nan dans note : 0
Valeurs nan dans date : 0
Valeurs nan dans contenu_complet : 0
Valeurs nan dans contenu_sans_spoils : 0

Valeurs uniques dans id_movie : 1
Valeurs uniques dans pseudo : 207
Valeurs uniques dans note : 10
Valeurs uniques dans date : 31
Valeurs uniques dans contenu_complet : 207
Valeurs uniques dans contenu_sans_spoils : 207
CPU times: total: 1.27 s
Wall time: 10.5 s


,id_movie,pseudo,note,date,contenu_complet,contenu_sans_spoils
0,281293,traversay1,"4,5",7 novembre 2022,"\nAprès le triomphe de Three Billboards, voir ...","\nAprès le triomphe de Three Billboards, voir ..."
1,281293,AZZZO,"5,0",11 janvier 2023,"\nDans ses ""Pensées"", Blaise Pascal écrivit qu...","\nDans ses ""Pensées"", Blaise Pascal écrivit qu..."
2,281293,Jorik V,"4,5",29 octobre 2022,\nCette œuvre est une petite pépite. Un diaman...,\nCette œuvre est une petite pépite. Un diaman...
3,281293,Benito G,"3,5",27 décembre 2022,\nMon Dieu que ce Banshees of Inisherin est dr...,\nMon Dieu que ce Banshees of Inisherin est dr...
4,281293,Cinéphiles 44,"4,5",28 décembre 2022,"\nDans les somptueux décors d'Irlande, le réal...","\nDans les somptueux décors d'Irlande, le réal..."


# Algorithme final 

In [29]:
%%time 
num_pages = 7 #Nombre maximal de pages scrappées par film 
nb_threads =5
list_ids =df_home[~df_home["note_moyenne_spectateurs"].isna()].sample(frac = 1)["id"].tolist()
nb_movies= len(list_ids)

ids_movies_by_threads = init_movies_by_driver(nb_threads, nb_movies)
init_save_files(nb_threads)

with futures.ThreadPoolExecutor() as executor: 
    future_results = [ executor.submit(workload_threads,{'id':i, 'index_start' :ids_movies_by_threads[i]["start"] , 'index_end':ids_movies_by_threads[i]["end"], 'num_pages':num_pages})  for i in range(nb_threads)] 
    for future_result in future_results: 
        try: 
            future_result = future_result.result()
        except Exception as exc: # can give a exception in some thread, but 
            print("thread generated an exception",exc)
            break;
        
#Concaténation et sauvegarde des données 
df=pd.DataFrame()
for i in range(nb_threads): 
    df=pd.concat([df, pd.read_csv(data_path+f"allocine_reviews/reviews_{i}.csv")])
df.to_csv(data_path+f"allocine_reviews.csv",index=False) 

Threads n°0 : 3950 films sur 3954
Threads n°1 : 3940 films sur 3954
Threads n°3 : 3940 films sur 3954
Threads n°1 : 3950 films sur 3954
Threads n°3 : 3950 films sur 3954
CPU times: total: 2h 4min 9s
Wall time: 3h 38min 44s


# Lecture des données 

In [30]:
df=pd.read_csv(data_path+f"allocine_reviews.csv")
print("Shape df_movies :",df.shape)
print()
check_nan(df)
print()
check_unique(df)
df.head()

Shape df_movies : (924876, 6)

Valeurs nan dans id_movie : 0
Valeurs nan dans pseudo : 180339
Valeurs nan dans note : 0
Valeurs nan dans date : 0
Valeurs nan dans contenu_complet : 0
Valeurs nan dans contenu_sans_spoils : 0

Valeurs uniques dans id_movie : 19058
Valeurs uniques dans pseudo : 55724
Valeurs uniques dans note : 10
Valeurs uniques dans date : 5941
Valeurs uniques dans contenu_complet : 920808
Valeurs uniques dans contenu_sans_spoils : 920711


,id_movie,pseudo,note,date,contenu_complet,contenu_sans_spoils
0,231865,AZZZO,"4,5",27 avril 2018,\nLa mort d'un fils. Samuel Maoz part de ce th...,\nLa mort d'un fils. Samuel Maoz part de ce th...
1,231865,traversay1,"4,5",27 avril 2018,\nDans cette vieille danse surannée qu'est le ...,\nDans cette vieille danse surannée qu'est le ...
2,231865,vidalger,"4,5",28 avril 2018,"\nAprès Lebanon, chef d'œuvre absolu sur la gu...","\nAprès Lebanon, chef d'œuvre absolu sur la gu..."
3,231865,velocio,"4,0",14 avril 2018,"\nIl y a 8 ans, le réalisateur israélien Samue...","\nIl y a 8 ans, le réalisateur israélien Samue..."
4,231865,Daniel C.,"5,0",28 avril 2018,\nQue signifie mourir ? Comment supporter la p...,\nQue signifie mourir ? Comment supporter la p...


# Nettoyage des données 

## Drop des dupliqués

In [32]:
df =df.drop_duplicates()

## Parsing des dates 

In [33]:
df['date'] = df['date'].apply(dateparser.parse)
df.head()

,id_movie,pseudo,note,date,contenu_complet,contenu_sans_spoils
0,231865,AZZZO,"4,5",2018-04-27,\nLa mort d'un fils. Samuel Maoz part de ce th...,\nLa mort d'un fils. Samuel Maoz part de ce th...
1,231865,traversay1,"4,5",2018-04-27,\nDans cette vieille danse surannée qu'est le ...,\nDans cette vieille danse surannée qu'est le ...
2,231865,vidalger,"4,5",2018-04-28,"\nAprès Lebanon, chef d'œuvre absolu sur la gu...","\nAprès Lebanon, chef d'œuvre absolu sur la gu..."
3,231865,velocio,"4,0",2018-04-14,"\nIl y a 8 ans, le réalisateur israélien Samue...","\nIl y a 8 ans, le réalisateur israélien Samue..."
4,231865,Daniel C.,"5,0",2018-04-28,\nQue signifie mourir ? Comment supporter la p...,\nQue signifie mourir ? Comment supporter la p...


# Sauvegarde

In [35]:
df.to_csv(data_path+"allocine_reviews_cleaned.csv",index=False)